In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import scipy as sp
# import contextily as cx

import torch
import pygsp
import optuna
import joblib
import gc
import argparse
import os
import matplotlib
import pickle
import glob

from matplotlib.ticker import ScalarFormatter, StrMethodFormatter, FormatStrFormatter, FuncFormatter
from matplotlib.animation import FuncAnimation

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from optuna.samplers import TPESampler, BruteForceSampler
from torch.nn import Linear
from torch_geometric.nn.models import GraphUNet
from torch_geometric.nn import GCNConv, Sequential
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx, grid

import sensors.utils.fault_detection as fd

from pyprojroot import here
ROOT_DIR = str(here())
data_dir = '/Users/vitorro/Repositories/dario/data/interim/'

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'Times New Roman'})

In [ ]:
# files = glob.glob(data_dir+"*")
# for file in sorted(files):
#     print(file)

In [ ]:
dataset = 'df_Gjerdrum_D1L2B'
df_orig = pd.read_parquet(data_dir + f'{dataset}.parq')
df_orig
print(df_orig.pid.nunique())

In [ ]:
df_metrics = fd.graph_filter(df_orig, cluster=True)

In [ ]:
anomalous_pixels = fd.detection(df_metrics, column_name='hf', quantile=0.995, detect_ratio=0.6)


In [ ]:
df = df_orig.copy()
df = df[~df.pid.isin(anomalous_pixels)]
df = df.merge(df_metrics[['hf']], how='left',left_index=True, right_index=True)
df = df.fillna(0)
df.to_parquet(data_dir + f'{dataset}_filtered.parq')

In [ ]:
fd.plot_selected_map(df_metrics, column_name='hf', size='hf', selected_pixels=anomalous_pixels, only_relevant=False,
                     figsize=(1600,900))

In [ ]:
df_metrics.pid.nunique()

In [ ]:
df_orig.pid.nunique()